# Interactive Data Visualization Using Matplotlib #

Useful link:

* [Matplotlib docs](https://matplotlib.org/stable/api/index)

First, some useful definitions:

In [ ]:
import datetime
import apsw as sqlite # docs: <https://rogerbinns.github.io/apsw/>

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import matplotlib.collections

from ipywidgets.widgets import \
    interact
import ipywidgets.widgets as \
    widgets

def db_iter(db, cmd, mapfn = lambda x : x) :
    "executes cmd on a new cursor from connection db and" \
    " yields the results in turn."
    cu = db.cursor()
    result = cu.execute(cmd)
    while True :
        try :
            yield mapfn(next(result))
        except StopIteration :
            break
        #end try
    #end while
#end db_iter

minutes_per_day = 1440

db_name = "/home/ldo/projects/ian_verne_stats/hack/data.db"

# matplotlib.rcParams["datapath"] = "/usr/share/matplotlib/sample_data/"

Load the data:

In [ ]:
interval_minutes = 15

db = sqlite.Connection(db_name, flags = sqlite.SQLITE_OPEN_READONLY)

unit_serials = list \
  (
    db_iter
      (
        db = db,
        cmd = "select distinct unit_serial from data order by unit_serial",
        mapfn = lambda x : x[0]
      )
  )
minute_buckets = set \
  (
    (day_nr * minutes_per_day + minute_nr) // interval_minutes
    for day_nr, minute_nr in
        db_iter
          (
            db = db,
            cmd = "select distinct day_nr, minute_nr from data"
          )
  )
minute_buckets_range = range(min(minute_buckets), max(minute_buckets) + 1)
print(minute_buckets_range) # debug

plot_data = [[0] * len(minute_buckets_range) for i in range(len(unit_serials))]
for u, unit_serial in enumerate(unit_serials) :
    cur_minute_bucket = None
    cur_usage = None
    entries = \
        db_iter \
          (
            db = db,
            cmd =
                    "select day_nr, minute_nr, usage from data"
                    " where usage is not null and unit_serial = %s"
                    " order by day_nr, minute_nr"
                %
                    sqlite.format_sql_value(unit_serial)
          )
    while True :
        entry = next(entries, None)
        if entry != None :
            day_nr, minute_nr, usage = entry
            minute_nr += day_nr * minutes_per_day
            minute_bucket = minute_nr // interval_minutes
        #end if
        if entry == None or minute_bucket != cur_minute_bucket :
            if cur_usage != None :
                plot_data[u][minute_bucket - min(minute_buckets_range)] = cur_usage
                cur_usage = None
            #end if
            if entry != None :
                cur_minute_bucket = minute_bucket
                cur_usage = 0
            #end if
        #end if
        if entry == None :
            break
        cur_usage += usage
    #end while
#end for
db.close()
if False :
    # debug -- smaller subset of data for easy display
    subrange = range(min(minute_buckets_range) + minutes_per_day // interval_minutes, min(minute_buckets_range) + minutes_per_day // interval_minutes + 60)
    plot_data = [row[min(subrange) - min(minute_buckets_range) : max(subrange) - min(minute_buckets_range) + 1] for row in plot_data]
    minute_buckets_range = subrange
    print(plot_data)
#end if

Common stuff for plots:

In [ ]:
def format_minutes(tickval, tickpos) :
    cur_minute = tickval * interval_minutes
    return \
      (
        (datetime.datetime(1970, 1, 1) + datetime.timedelta(minutes = cur_minute))
            .strftime("%b-%d %H:%M")
      )
#end format_minutes

def cc(arg):
    return matplotlib.colors.to_rgba(arg, alpha = 0.6)
#end cc

def format_minutes(tickval, tickpos) :
    cur_minute = tickval * interval_minutes
    return \
      (
        (datetime.datetime(1970, 1, 1) + datetime.timedelta(minutes = cur_minute))
            .strftime("%b-%d %H:%M")
      )
#end format_minutes

def format_units(tickval, tickpos) :
    tickval = float(tickval)
      # avoid warnings about using numpy bools as indexes
    return \
        (
            lambda : "",
            lambda : "%d" % unit_serials[int(tickval)],
        )[tickval == int(tickval) and 0 <= tickval and tickval < len(unit_serials)]()
#end format_units


[Wire3d](https://matplotlib.org/stable/gallery/mplot3d/wire3d.html) plot:

In [ ]:
@interact(elev = (-90, 90, 10), azi = (-90, 90, 10))
def plotit(elev, azi) :
    fig = plt.figure(figsize = (10, 12))
    # docs: <https://matplotlib.org/stable/api/toolkits/mplot3d/faq.html>
    ax = fig.add_subplot(111, projection = "3d")
    ax.margins(0, 0, 0)
    X = np.array([[u] * len(minute_buckets_range) for u in range(len(unit_serials))])
    Y = np.array([list(minute_buckets_range)] * len(unit_serials))
    Z = np.array(plot_data)
    # print("X = ", list(len(x) for x  in X), X) # debug
    # print("Y = ", list(len(y) for y in Y), Y) # debug
    # print("Z = ", list(len(z) for z in Z), Z) # debug
    ax.plot_wireframe(X, Y, Z)
    ax.set_zlabel('usage')
    ax.set_xlabel('unit')
    #ax.set_xlim3d(-1, len(unit_serials))
    ax.xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_units))
    ax.set_ylim3d(min(minute_buckets_range), max(minute_buckets_range))
    ax.set_ylabel('minute')
    ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_minutes))
    ax.view_init(elev, azi)

    plt.show()
#end plotit

[Polys3d](https://matplotlib.org/stable/gallery/mplot3d/polys3d.html) plot:

In [ ]:
@interact(elev = (-90, 90, 10), azi = (-90, 90, 10))
def plotit(elev, azi) :
    fig = plt.figure(figsize = (10, 12))
    #ax = fig.gca(projection = '3d')
    ax = fig.add_axes((0, 0, 1, 1), projection = '3d')
    verts = \
        [
            [
                (m, plot_data[u][m - min(minute_buckets_range)])
                for m in minute_buckets_range
            ]
            for u in range(len(unit_serials))
        ]
    for row in verts :
        row[0] = (row[0][0], 0)
        row[-1] = (row[-1][0], 0)
    #end for
    #print(verts)
    poly = matplotlib.collections.PolyCollection(verts, facecolors=[cc('r'), cc('g'), cc('b')])
    poly.set_alpha(0.7)
    ax.add_collection3d(poly, zs = list(range(len(unit_serials))), zdir = "x")
    ax.set_xlim3d(-1, len(unit_serials))
    ax.set_zlim3d(0, max(max(c[1] for c in row) for row in verts))
    ax.set_zlabel('usage')
    ax.set_xlabel('unit')
    #ax.set_xlim3d(-1, len(unit_serials))
    ax.xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_units))
    ax.set_ylim3d(min(minute_buckets_range), max(minute_buckets_range))
    ax.set_ylabel('minute')
    ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_minutes))
    ax.view_init(elev, azi)

    plt.show()
#end plotit